# Machine Learning - Project 1

In [1]:
from mlcomp.data import load_csv_data

In [2]:
# Setup and imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

import proj1_helpers

## Implementations of ML Methods

### Test data:

Testing data for helper functions

In [3]:
y = np.array([1, 2, 3, 4])
tx = np.array([[1, 2, 3], 
               [4, 5, 6], 
               [7, 8, 9], 
               [10, 11, 12]])
w = np.array([0, 0, 0])

### Helper functions:

In [4]:
def compute_loss_mse(y, tx, w):
    """Calculate the MSE loss."""
    N = len(y)
    e = y - np.dot(tx, w)
    return np.dot(e.T,e) / (2 * N)

In [5]:
def compute_loss_mae(y, tx, w):
    """Calculate the MAE loss."""
    N = len(y)
    e = y - np.dot(tx, w)
    return np.sum(np.absolute(e)) / N

In [6]:
def compute_rmse(y, tx, w):
    """Computes the Root Mean Square Error"""
    mse = compute_loss_mse(y, tx, w)
    return np.sqrt(2 * mse)

In [7]:
def compute_gradient_mse(y, tx, w):
    """Compute the MSE gradient."""
    N = len(y)
    e = y - np.dot(tx, w)
    return (-1/N) * np.dot(np.transpose(tx), e)

In [8]:
def compute_stochastic_subgradient_mae(y, tx, w):
    """Compute a stochastic subgradient from just few examples n and their corresponding y_n labels."""
    N = len(y)
    e = y - np.dot(tx, w)
    abs_e_subgrad = [np.sign(en) for en in e] # Sign chosen for subgradient of absolute value function
    return (-1/N) * np.dot(np.transpose(tx), abs_e_subgrad)

In [9]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [10]:
def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """
    np.random.seed(seed) # set seed
    permuted_idxs = np.random.permutation(x.shape[0])
    train_size = int(ratio * x.shape[0])
    train_idxs, test_idxs = permuted_idxs[:train_size], permuted_idxs[train_size:]
    
    return x[train_idxs], x[test_idxs], y[train_idxs], y[test_idxs]

In [11]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [12]:
def get_train_indices(k_indices, k):
    train_indices = np.array([])
    for i in range(k_indices.shape[0]):
        if (i != k-1):
            train_indices = np.hstack((train_indices, k_indices[i]))
    return train_indices.astype(int)

### Functions to implement for project 1 submission:

In [13]:
"""Linear regression using gradient descent"""
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm using MSE."""
    w = initial_w
    for n_iter in range(max_iters):
        grad = compute_gradient_mse(y, tx, w)
        loss = compute_loss_mse(y, tx, w)
        w = w - gamma * grad
        
    rmse = compute_rmse(y, tx, w)

    return w, rmse

In [14]:
"""Linear regression using stochastic gradient descent"""
def stochastic_subgradient_descent_mae(y, tx, initial_w, max_iters, gamma, batch_size):
    """Stochastic subgradient descent algorithm using MAE."""
    w = initial_w
    for n_iter in range(max_iters):
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            g = compute_stochastic_subgradient_mae(minibatch_y, minibatch_tx, w)
            w = w - gamma * g
        loss = compute_loss_mae(y, tx, w)
        print("Stochastic Subgradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return w, loss

In [15]:
def least_squares(y, tx):
    """Calculates the explicit least squares solution.
    Returns rmse, optimal weights"""
    N = tx.shape[0]
    D = tx.shape[1]
    rank_tx = np.linalg.matrix_rank(tx)
    
    # Check if tx is invertible. If so, find explicit solution
    # using real inverses.
    # If not, find explicit solution using pseudoinverses.
    if (rank_tx == max(tx.shape[0], tx.shape[1])):
        gramian_inv = np.linalg.solve(np.dot(tx.T, tx), np.identity(D))
        w = np.dot(gramian_inv, np.dot(tx.T, y))
    else:
        U, s, V_T = np.linalg.svd(tx)
        S_inv_T = np.zeros((D, N))
        S_inv_T[:len(s), :len(s)] = np.diag(1/s)
        w = np.dot(V_T.T, np.dot(S_inv_T, np.dot(U.T, y)))
    
    rmse = compute_rmse(y, tx, w)
    
    return w, rmse

In [16]:
def ridge_regression(y, tx, lambda_):
    """Ridge regression using normal equations"""
    
    #if (lambda_ == 0):
    #    return least_squares(y, tx)
    
    N = tx.shape[0]
    D = tx.shape[1]
    
    inv_inner = np.dot(tx.T, tx) + 2 * N * lambda_ * np.identity(D)
    inv = np.linalg.solve(inv_inner, np.identity(D))
    w = np.dot(inv, np.dot(tx.T, y))
    
    rmse = compute_rmse(y, tx, w)
    
    return w, rmse

In [17]:
"""Logistic regression using gradient descent or SGD"""
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    raise NotImplementedError

In [18]:
"""Regularized logistic regression using gradient descent or SGD"""
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    raise NotImplementedError

## Higgs Data Analysis

### Experimental functions

All of these functions can be used experimentally to, in one way or the other, improve the predictions.

In [19]:
def correctness(yb, y_pred):
    """Takes inputs known y and predicted y and prints the ratio of correct predictions vs incorrect ones."""
    correct = 0
    for i in range(len(y_pred)):
        if (y_pred[i] == yb[i]):
            correct += 1
        
    incorrect = len(y_pred) - correct
    perc = correct / len(y_pred) * 100
    print("Total correct:", correct, "\nTotal incorrect:", incorrect, "\nCorrect percentage:", perc, "%")

In [20]:
def replace_nan_by_mean(tx, nan_value):
    """Replaces values with a specified nan_value by the column mean."""
    tx[tx == nan_value] = np.nan
    col_mean = np.nanmean(tx, axis=0)
    return np.where(np.isnan(tx), col_mean, tx)

In [21]:
def replace_nan_by_median(tx, nan_value):
    """Replaces values with a specified nan_value by the column median."""
    tx[tx == nan_value] = np.nan
    col_median = np.nanmedian(tx, axis=0)
    return np.where(np.isnan(tx), col_median, tx)

In [22]:
def drop_nan_rows(tx, nan_value):
    """Drop all rows that contain a nan equaling the specified nan_value."""
    tx[tx == nan_value] = np.nan
    return tx[~np.isnan(tx).any(axis=1)]

In [23]:
def build_simple_poly(tx, degree):
    """
    Builds simple polynomial basis function for input data matrix tx, for j=0 up to j=degree,
    where the result will be a matrix of form [1, tx, tx^2, ..., tx^j].
    tx^j denotes that for each x_i,k in tx, the result will be (x_i,k)^j
    """
    poly = np.ones((tx.shape[0], 1))

    for j in range(1, degree+1):
        poly = np.column_stack((poly, np.power(tx, j)))

    return poly

In [24]:
import itertools

# TODO: Optimize. Very slow!
def build_mult_comb(tx, deg, cols=[]):
    """
    Returns all multiplicative combinations of the specified columns for degree deg.
    For len(col) = D', there are (D' choose deg) combinations of columns that get
    returned as a matrix.
    If cols is not given, it returns the combinations of all columns of tx.
    """
    N = tx.shape[0]
    if (cols == []):
        comb_iter = itertools.combinations_with_replacement(range(tx.shape[1]), deg)
    else:
        comb_iter = itertools.combinations_with_replacement(cols, deg)
    mult = []
    for comb in comb_iter:
        mult_col = np.ones(N)
        for idx in comb:
            tx_col = tx[:,idx]
            mult_col = np.multiply(mult_col, tx_col)
        mult.append(mult_col.tolist())
    return np.array(mult).T

def build_advanced_poly(tx, degree, cols=[]):
    """
    Builds full polynomial basis function for input data matrix tx, for j=0 up to j=degree,
    where the result will be a matrix of form:
    [1, tx, comb_mult(tx, 2), ..., comb_mult(tx, j)]
    comb_mult(tx, 2) denotes all multiplicative combinations of the selected columns of tx.
    If cols is not given, it returns the combinations of all columns of tx.
    """
    poly = np.ones((tx.shape[0], 1))

    for j in range(1, degree+1):
        mult = build_mult_comb(tx, j, cols)
        poly = np.column_stack((poly, mult))

    return poly

# Testing on data

Import training data:

In [25]:
from proj1_helpers import *

data_path = "../data/train.csv"
yb, input_data, ids = load_csv_data(data_path, sub_sample=False)
yb = yb.reshape((yb.shape[0], 1))

In [26]:
print(yb.shape, input_data.shape)

(250000, 1) (250000, 30)


Transform the data to more useable data, by replacing NaN's by the column mean and creating a polynomial base expansion.

By previous data analysis, important_cols are selected to be the few "most influencial" features. We use only those few weights in the polynomial base expansion for computational efficiency.

| Index | Feature                   |
|-------|---------------------------|
|  0    | DER_mass_MMC              |
| 1     | DER_mass_traverse_met_lep |
| 2     | DER_mass_vis              |
| 13    | PRI_tau_pt                |
| 11    | DER_met_phi_centrality    |
| 10    | DER_pt_ratio_lep_tau      |
| 7     | DER_deltar_tau_lep        |
| 19    | PRI_met                   |

In [27]:
input_data = replace_nan_by_median(input_data, -999)

important_cols = [0, 1, 2, 7, 10, 11, 13, 19]
#important_cols = [0, 1, 2, 10, 11, 13]
degree = 2 # With current implementation, higher than 5 is comp. infeasable, but it gives the best results!

x_poly = build_advanced_poly(input_data, degree, important_cols)

x_train, x_test, y_train, y_test = split_data(x_poly, yb, ratio=0.9, seed=123456789)

In [28]:
initial_w = np.zeros((x_train.shape[1], 1))
max_iters = 100
gamma = 1
batch_size = 1

In [29]:
weights, loss = stochastic_subgradient_descent_mae(y_train, x_train, initial_w, max_iters, gamma, batch_size)

Stochastic Subgradient Descent(0/99): loss=233339751.11872205, w0=[-1.], w1=[-77.112]
Stochastic Subgradient Descent(1/99): loss=72292625.66239856, w0=[ 0.], w1=[ 21.414]
Stochastic Subgradient Descent(2/99): loss=271285380.54342985, w0=[-1.], w1=[-73.125]
Stochastic Subgradient Descent(3/99): loss=91942031.41665281, w0=[ 0.], w1=[ 25.157]
Stochastic Subgradient Descent(4/99): loss=669453570.8104852, w0=[-1.], w1=[-123.896]
Stochastic Subgradient Descent(5/99): loss=151155430.155957, w0=[ 0.], w1=[-11.49]
Stochastic Subgradient Descent(6/99): loss=493021424.5928208, w0=[ 1.], w1=[ 119.291]
Stochastic Subgradient Descent(7/99): loss=66687581.32247934, w0=[ 0.], w1=[ 12.542]
Stochastic Subgradient Descent(8/99): loss=560973129.1062058, w0=[-1.], w1=[-95.854]
Stochastic Subgradient Descent(9/99): loss=88444150.82411285, w0=[ 0.], w1=[ 20.846]
Stochastic Subgradient Descent(10/99): loss=979620421.6364462, w0=[ 1.], w1=[ 184.794]
Stochastic Subgradient Descent(11/99): loss=380819345.6720501

Stochastic Subgradient Descent(95/99): loss=810363326.8294797, w0=[-6.], w1=[-549.689]
Stochastic Subgradient Descent(96/99): loss=380686911.1463355, w0=[-5.], w1=[-429.427]
Stochastic Subgradient Descent(97/99): loss=459412666.15058506, w0=[-4.], w1=[-317.021]
Stochastic Subgradient Descent(98/99): loss=334178281.1111742, w0=[-5.], w1=[-456.971]
Stochastic Subgradient Descent(99/99): loss=244281560.8211903, w0=[-4.], w1=[-354.641]


In [31]:
# Predict labels with found weights and print some useful information about quality of fit
y_pred = predict_labels(weights, x_poly)
correctness(yb, y_pred)

Total correct: 152532 
Total incorrect: 97468 
Correct percentage: 61.0128 %


Load and transform test data:

In [ ]:
test_path = "../data/test.csv"
yb_test, input_data_test, ids_test = load_csv_data(test_path, sub_sample=False)

In [ ]:
input_data_test = replace_nan_by_median(input_data_test, -999)
x_submit_poly = build_advanced_poly(input_data_test, degree, important_cols)

In [ ]:
y_test_pred = predict_labels(weights, x_submit_poly)

In [ ]:
# Save predictions of test data in csv file, ready for the upload on kaggle
create_csv_submission(ids_test, y_test_pred, "test_output.csv")